In [1]:
from osgeo import gdal, ogr, gdalconst
from pathlib import Path
import plotly.express as px
import cv2
import imageio

In [2]:
prodes_folder = Path(r'D:\Ferrari\Projects\deforestation-forescast\data\deter')

In [3]:
base_path = prodes_folder / 'base.tif'
base_data = gdal.Open(str(base_path), gdalconst.GA_ReadOnly)

geo_transform = base_data.GetGeoTransform()
x_res = base_data.RasterXSize
y_res = base_data.RasterYSize
crs = base_data.GetSpatialRef()
proj = base_data.GetProjection()

In [4]:
mask = prodes_folder / 'brazilian_legal_amazon.shp'
mask_v = ogr.Open(str(mask))
mask_l = mask_v.GetLayer()

In [5]:
mask_file = Path(r'D:\Ferrari\Projects\deforestation-forescast\data\deter\area.tif')

mask_data = gdal.GetDriverByName('GTiff').Create(str(mask_file), x_res, y_res, 1, gdal.GDT_Byte)
mask_data.SetGeoTransform(geo_transform)
mask_data.SetSpatialRef(crs)
mask_data.SetProjection(proj)

gdal.RasterizeLayer(mask_data, [1], mask_l, burn_values=[1], options = ['ALL_TOUCHED=TRUE'])

mask_data = None

In [6]:
train_years = [2017, 2018, 2019]
test_years = [2020, 2021, 2022]

In [7]:
deter_path = prodes_folder / 'deter-amz-deter-public.shp'
deter_v = ogr.Open(str(deter_path))
deter_l = deter_v.GetLayer()

## Training data

In [8]:
train_file = prodes_folder / 'train.tif'
train_data = gdal.GetDriverByName('GTiff').Create(str(train_file), x_res, y_res, 12*len(train_years), gdal.GDT_Byte)
train_data.SetGeoTransform(geo_transform)
train_data.SetSpatialRef(crs)
train_data.SetProjection(proj)

0

In [9]:
count_band = 1
captions = []
for year in train_years:
    for month in range(1, 13):
        where = f' "year" = {year} and  "month" = {month}'
        deter_l.SetAttributeFilter(where)
        gdal.RasterizeLayer(train_data, [count_band], deter_l, burn_values=[1], options = ['ALL_TOUCHED=TRUE'])
        
        count_band+=1
        captions.append(f'{year}/{month}')

deter_array = train_data.ReadAsArray() 
train_data = None

In [10]:
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (2550, 2300) 
fontScale = 3

color = (1, 0, 0) 
thickness = 4

for i, frame in enumerate(deter_array):
    image = cv2.putText(frame, captions[i], org, font,  
                    fontScale, color, thickness, cv2.LINE_AA) 

In [11]:
fig = px.imshow(deter_array, animation_frame=0,  binary_string=True)
fig.show()

In [12]:
gif_file = prodes_folder / 'train.gif'
print("Saving GIF file")
with imageio.get_writer(gif_file, mode="I", fps = 1) as writer:
    for idx, frame in enumerate(deter_array):
        print("Adding frame to GIF file: ", idx + 1)
        writer.append_data(frame*255)

Saving GIF file
Adding frame to GIF file:  1
Adding frame to GIF file:  2
Adding frame to GIF file:  3
Adding frame to GIF file:  4
Adding frame to GIF file:  5
Adding frame to GIF file:  6
Adding frame to GIF file:  7
Adding frame to GIF file:  8
Adding frame to GIF file:  9
Adding frame to GIF file:  10
Adding frame to GIF file:  11
Adding frame to GIF file:  12
Adding frame to GIF file:  13
Adding frame to GIF file:  14
Adding frame to GIF file:  15
Adding frame to GIF file:  16
Adding frame to GIF file:  17
Adding frame to GIF file:  18
Adding frame to GIF file:  19
Adding frame to GIF file:  20
Adding frame to GIF file:  21
Adding frame to GIF file:  22
Adding frame to GIF file:  23
Adding frame to GIF file:  24
Adding frame to GIF file:  25
Adding frame to GIF file:  26
Adding frame to GIF file:  27
Adding frame to GIF file:  28
Adding frame to GIF file:  29
Adding frame to GIF file:  30
Adding frame to GIF file:  31
Adding frame to GIF file:  32
Adding frame to GIF file:  33
Add

In [13]:
window_name = 'Image'
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (2650, 2300) 
fontScale = 3
   
color = (255, 0, 0) 
thickness = 4
   
image = cv2.putText(deter_array[0], 'OpenCV', org, font,  
                   fontScale, color, thickness, cv2.LINE_AA) 
   

In [14]:
fig = px.imshow(image, binary_string=True)
fig.show()